#### Author
Yury Kashnitsky

#### Reference

#### Idea
Process all labeled data at hand to insert into the `news_titles` and `labeled_news_titles` tables.

#### Data

We are resuing data reading utils written by Victor in [here](https://github.com/crypto-sentiment/crypto_sentiment_notebooks/pull/10). It merges 4500 titles labeled in 2017-2018 with 400 titles labeled by us.

#### Result

Two files to be imported to tables `news_titles` and `labeled_news_titles`:

- ../data/20220606_news_titles_to_import.csv
- ../data/20220606_labeled_news_titles_to_import.csv

In [1]:
import pandas as pd
from mmh3 import hash as mmh3_hash
from checklist_utils.data import read_data, read_assessment_data

In [2]:
LABEL_MAP =  {"Negative": 0, "Neutral": 1, "Positive": 2}
SEED = 17

Freshly labeled data

In [3]:
df_new = read_assessment_data(data_path='../data/')

In [4]:
len(df_new)

392

In [5]:
df_new.head(2)

,title,sentiment
0,"""Not as Anonymous as People Think,"" Shows DOJ ...",Positive
1,'Bitcoin Centrist' Muneeb Ali: Why NFTs Are Tr...,Neutral


In [6]:
df = read_data(data_path='../data/')

In [7]:
len(df)

4758

In [8]:
df.head(2)

,title,sentiment,label
400,Burger King in Russia Enable Bitcoin Payments ...,Positive,2
3145,Bitcoin Mining Giant Bitmain to Release Next-G...,Positive,2


We miss metadata but at least the freshly labeled data was annotated in April 2022, while the old data was annotated around January 2019. So we fill in the values accordingly. 

In [9]:
df['annot_time'] = '2019-01-01'
df.loc[df_new.index, 'annot_time'] = '2022-04-01'

In [10]:
df['label'] = df['sentiment'].map(LABEL_MAP)

In [11]:
df['title_id'] = df['title'].apply(lambda s: mmh3_hash(s, seed=SEED))

In [12]:
df.head()

,title,sentiment,label,annot_time,title_id
400,Burger King in Russia Enable Bitcoin Payments ...,Positive,2,2019-01-01,-112618357
3145,Bitcoin Mining Giant Bitmain to Release Next-G...,Positive,2,2019-01-01,-451501667
2241,"Acid Test: Bitcoin Must Break $7,800 for Bull ...",Positive,2,2019-01-01,36355323
2056,Blockchain Entrepreneurs Invigorated By CMEs B...,Positive,2,2019-01-01,-49025870
3009,Irans Bitcoin Volume Soars as Rial Value Enter...,Positive,2,2019-01-01,1535460804


We have some historic metadata, let's try to get URLs from there

In [13]:
metadata_df = pd.concat([
    pd.read_csv('../data/archive/crypto_news_parsed_2013-2017_train.csv.zip'),
    pd.read_csv('../data/archive/crypto_news_parsed_2018_validation.csv.zip'),
    pd.read_csv('../data/archive/btc_titles_17-18.csv')
]).drop_duplicates(subset=['title'])

In [14]:
metadata_df.head(2)

,url,title,text,html,year,author,source,cleaned_title,date,num_words
0,https://www.ccn.com/bitcoin-price-update-will-...,Bitcoin Price Update: Will China Lead us Down?,Bitcoin Priced in USD on Mt. GoxAbove is Gox p...,"<figure id=""attachment_4090"" style=""width: 838...",2013.0,Gordon Hall,altcoin_analysis,NaN,NaN,NaN
1,https://www.ccn.com/key-bitcoin-price-levels-w...,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,"The Bitcoin price up to 2013/12/14, with Fib l...","<figure id=""attachment_3961"" style=""width: 838...",2013.0,Gordon Hall,altcoin_analysis,NaN,NaN,NaN


In [15]:
df_with_metadata = df[['title_id', 'title', 'label', 'annot_time']].merge(
    metadata_df[['title', 'url', 'date']], on='title', how='left')

In [16]:
df_with_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4758 entries, 0 to 4757
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title_id    4758 non-null   int64 
 1   title       4758 non-null   object
 2   label       4758 non-null   int64 
 3   annot_time  4758 non-null   object
 4   url         3209 non-null   object
 5   date        492 non-null    object
dtypes: int64(2), object(4)
memory usage: 260.2+ KB


We fill in sources (URLs) with "missing", and publication dates – with a random date from 2017-2018

In [17]:
from random import randrange
from datetime import timedelta, datetime

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [18]:
df_with_metadata['source'] = df_with_metadata['url'].fillna('missing')
df_with_metadata['pub_time'] = [random_date(start=datetime.strptime('2017-01-01', "%Y-%m-%d"),
            end=datetime.strptime('2018-12-31', "%Y-%m-%d")).strftime("%Y-%m-%d")
                               for _ in range(len(df_with_metadata))]

In [19]:
df_with_metadata = df_with_metadata[['title_id', 'title', 'source', 'pub_time', 'label', 'annot_time']]

In [20]:
df_with_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4758 entries, 0 to 4757
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title_id    4758 non-null   int64 
 1   title       4758 non-null   object
 2   source      4758 non-null   object
 3   pub_time    4758 non-null   object
 4   label       4758 non-null   int64 
 5   annot_time  4758 non-null   object
dtypes: int64(2), object(4)
memory usage: 260.2+ KB


In [21]:
df_with_metadata.head(2)

,title_id,title,source,pub_time,label,annot_time
0,-112618357,Burger King in Russia Enable Bitcoin Payments ...,https://www.newsbtc.com/2017/06/28/burger-king...,2018-11-03,2,2019-01-01
1,-451501667,Bitcoin Mining Giant Bitmain to Release Next-G...,missing,2017-03-13,2,2019-01-01


We need 'title_id', 'title', 'source', 'pub_time' for the `news_titles` table and 'title_id', 'label', 'annot_time' for the `labeled_news_titles` table. 

In [22]:
df_with_metadata[['title_id', 'title', 'source', 'pub_time']].to_csv('../data/20220606_news_titles_to_import.csv',
                                                                    index=None)
df_with_metadata[['title_id', 'label', 'annot_time']].to_csv('../data/20220606_labeled_news_titles_to_import.csv',
                                                                    index=None)